# Dataproject

In [2]:
### LAV TOTALER AF ALDER FOR BÅDE FOD407 OG AULP01



# importing crucial packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pydst
dst = pydst.Dst(lang='en')

In [33]:
## What does DST have to offer?

dst.get_subjects()

,active,desc,hasSubjects,id
0,True,Population and elections,True,02
1,True,Living conditions,True,05
2,True,Education and knowledge,True,03
3,True,Culture and National Church,True,18
4,True,"Labour, income and wealth",True,04
5,True,Prices and consumption,True,06
6,True,National accounts and government finances,True,14
7,True,Money and credit market,True,16
8,True,External economy,True,13
9,True,Business sector in general,True,07


We would like to look at "population and elections"

In [34]:
dst.get_tables(subjects=['02'])

,active,firstPeriod,id,latestPeriod,text,unit,updated,variables
0,True,2008Q1,FOLK1A,2019Q1,Population at the first day of the quarter,number,2019-02-11 08:00:00,"[region, sex, age, marital status, time]"
1,True,2008Q1,FOLK1B,2019Q1,Population at the first day of the quarter,number,2019-02-11 08:00:00,"[region, sex, age, citizenship, time]"
2,True,2008Q1,FOLK1C,2019Q1,Population at the first day of the quarter,number,2019-02-11 08:00:00,"[region, sex, age, ancestry, country of origin..."
3,True,2008Q1,FOLK1D,2019Q1,Population at the first day of the quarter,number,2019-02-11 08:00:00,"[region, sex, age, citizenship, time]"
4,True,2008Q1,FOLK1E,2019Q1,Population at the first day of the quarter,number,2019-02-11 08:00:00,"[region, sex, age, ancestry, time]"
5,True,1980,FOLK2,2019,Population 1. January,number,2019-02-11 08:00:00,"[age, sex, ancestry, citizenship, country of o..."
6,True,2008,FOLK3,2019,Population 1. January,number,2019-02-11 08:00:00,"[day of birth, birth month, year of birth, time]"
7,True,1769,FT,2019,Population figures from the censuses,number,2019-02-11 08:00:00,"[national part, time]"
8,True,2008,BEF5F,2019,People born in Faroe Islands and living in Den...,number,2019-02-11 08:00:00,"[sex, age, parents place of birth, time]"
9,True,2008,BEF5G,2019,People born in Greenland and living in Denmark...,number,2019-02-11 08:00:00,"[sex, age, parents place of birth, time]"


We would like to look at the table with id 'FODIE', lets look at the variables

In [41]:
FOD407_vars = dst.get_variables(table_id='FOD407')
FOD407_vars
FOD407_vars['values'][1][:] ## age= TOT1 to include all ages
FOD407_vars['values'][1][:] ##Women = 2



[{'id': 'TOT1', 'text': 'Total fertility rate'},
 {'id': '15-19', 'text': '15-19 years'},
 {'id': '20-24', 'text': '20-24 years'},
 {'id': '25-29', 'text': '25-29 years'},
 {'id': '30-34', 'text': '30-34 years'},
 {'id': '35-39', 'text': '35-39 years'},
 {'id': '40-44', 'text': '40-44 years'},
 {'id': '45-49', 'text': '45-49 years'}]

In [64]:
FOD407 = dst.get_data(table_id = 'FOD407', variables={'OMRÅDE':['*'], 'TID':['*'], 'ALDER':['*'] })
#testtest=FOD407.sort(['INDHOLD'])
FOD407.sort_values(by=['OMRÅDE'], ascending=False)

,OMRÅDE,TID,ALDER,INDHOLD
9198,Ærø,2015,30-34 years,..
11636,Ærø,2018,Total fertility rate,..
2973,Ærø,2009,20-24 years,..
5409,Ærø,2011,45-49 years,..
4017,Ærø,2010,25-29 years,..
11632,Ærø,2018,30-34 years,..
11633,Ærø,2018,35-39 years,..
11634,Ærø,2018,40-44 years,..
11635,Ærø,2018,45-49 years,..
537,Ærø,2006,35-39 years,..


In [60]:
FOD407_en=FOD407.rename(columns={"OMRÅDE": "AREA", "TID": "YEAR", "INDHOLD": "BIRTHS", "ALDER": "AGE"})
FOD407_en.head()

,AREA,YEAR,AGE,BIRTHS
0,All Denmark,2006,15-19 years,4.3
1,Region Hovedstaden,2006,15-19 years,4.0
2,Region Sjælland,2006,15-19 years,5.4
3,Region Syddanmark,2006,15-19 years,4.7
4,Region Midtjylland,2006,15-19 years,3.2


In [54]:
FOD407_en_group = FOD407_en.groupby(['AREA','YEAR'])['BIRTHS'].sum().reset_index()
FOD407_en_group.head()

,AREA,YEAR,BIRTHS
0,Aabenraa,2006,4.969.2159.1143.045.17.80.02146.1
1,Aabenraa,2007,7.159.5164.4120.347.16.00.02021.9
2,Aabenraa,2008,5.460.2149.0141.550.87.60.42075.0
3,Aabenraa,2009,4.365.9167.5143.047.97.00.02177.6
4,Aabenraa,2010,2.352.9154.0139.849.19.30.02037.3


In [57]:
FOD407_en_alder=FOD407_en.loc[(FOD407_en['AGE'] >= '15') & (FOD407_en['AGE'] <= '49')]
FOD407_en_alder.head()


##TROR IKKE AT DETTE STYKKE KODE ER NØDVENDIGT

,AREA,YEAR,AGE_FOD407,BIRTHS
0,All Denmark,2006,15-19 years,4.3
1,Region Hovedstaden,2006,15-19 years,4.0
2,Region Sjælland,2006,15-19 years,5.4
3,Region Syddanmark,2006,15-19 years,4.7
4,Region Midtjylland,2006,15-19 years,3.2


In [39]:
FOD407_alder_group = FOD407_en_alder.groupby(['AREA','YEAR'])['BIRTHS'].sum().reset_index()
FOD407_alder_group.head()

,AREA,YEAR,BIRTHS
0,Aabenraa,2007,596
1,Aabenraa,2008,611
2,Aabenraa,2009,622
3,Aabenraa,2010,571
4,Aabenraa,2011,540


We know look at "Labour, income and wealth"

In [4]:
dst.get_tables(subjects=['04'])

,active,firstPeriod,id,latestPeriod,text,unit,updated,variables
0,True,2008Q1,FOLK1A,2019Q1,Population at the first day of the quarter,number,2019-02-11 08:00:00,"[region, sex, age, marital status, time]"
1,True,2008Q1,FOLK1B,2019Q1,Population at the first day of the quarter,number,2019-02-11 08:00:00,"[region, sex, age, citizenship, time]"
2,True,2008Q1,FOLK1C,2019Q1,Population at the first day of the quarter,number,2019-02-11 08:00:00,"[region, sex, age, ancestry, country of origin..."
3,True,2008Q1,FOLK1D,2019Q1,Population at the first day of the quarter,number,2019-02-11 08:00:00,"[region, sex, age, citizenship, time]"
4,True,2008Q1,FOLK1E,2019Q1,Population at the first day of the quarter,number,2019-02-11 08:00:00,"[region, sex, age, ancestry, time]"
5,True,1980,FOLK2,2019,Population 1. January,number,2019-02-11 08:00:00,"[age, sex, ancestry, citizenship, country of o..."
6,True,2008,FOLK3,2019,Population 1. January,number,2019-02-11 08:00:00,"[day of birth, birth month, year of birth, time]"
7,True,1769,FT,2019,Population figures from the censuses,number,2019-02-11 08:00:00,"[national part, time]"
8,True,2008,BEF5F,2019,People born in Faroe Islands and living in Den...,number,2019-02-11 08:00:00,"[sex, age, parents place of birth, time]"
9,True,2008,BEF5G,2019,People born in Greenland and living in Denmark...,number,2019-02-11 08:00:00,"[sex, age, parents place of birth, time]"


In [5]:
AULP01_vars = dst.get_variables(table_id='AULP01')
AULP01_vars

,elimination,id,map,text,time,values
0,True,OMRÅDE,denmark_municipality_07,region,False,"[{'id': '000', 'text': 'All Denmark'}, {'id': ..."
1,True,ALDER,NaN,age,False,"[{'id': 'TOT', 'text': 'Age, total'}, {'id': '..."
2,True,KØN,NaN,sex,False,"[{'id': 'TOT', 'text': 'Total'}, {'id': 'M', '..."
3,False,Tid,NaN,time,True,"[{'id': '2007', 'text': '2007'}, {'id': '2008'..."


In [12]:
AULP01 = dst.get_data(table_id = 'AULP01', variables={'OMRÅDE':['*'], 'ALDER':['*'], 'KØN':['*'], 'TID':['*'] })
AULP01.head()

In [59]:
AULP01_en = AULP01.rename(columns={"OMRÅDE": "AREA", "ALDER": "AGE", "KØN": "GENDER",\
                                   "TID": "YEAR", "INDHOLD": "UNEMPLOYMENT RATE"})
AULP01_en.head()

,AREA,AGE,GENDER,YEAR,UNEMPLOYMENT RATE
0,Syddjurs,16-24 years,Total,2012,4.1
1,Syddjurs,25-29 years,Total,2012,9.7
2,Syddjurs,30-34 years,Total,2012,7.4
3,Syddjurs,35-39 years,Total,2012,5.5
4,Syddjurs,40-44 years,Total,2012,4.9


In [49]:
AULP01R_en_group = AULP01_en.groupby(['AREA','YEAR'])['UNEMPLOYMENT RATE'].sum().reset_index()
AULP01R_en_group.head()

,AREA,YEAR,UNEMPLOYMENT RATE
0,Aabenraa,2007,120.2
1,Aabenraa,2008,87.1
2,Aabenraa,2009,158.0
3,Aabenraa,2010,207.1
4,Aabenraa,2011,199.9


We now merge the two different datasets.

In [67]:
merged_data = pd.merge(FOD407_en, AULP01_en, on=['YEAR', 'AREA'], how='left')
merged_data.head(100)

,AREA,YEAR,AGE_x,BIRTHS,AGE_y,GENDER,UNEMPLOYMENT RATE
0,All Denmark,2006,15-19 years,4.3,NaN,NaN,NaN
1,Region Hovedstaden,2006,15-19 years,4.0,NaN,NaN,NaN
2,Region Sjælland,2006,15-19 years,5.4,NaN,NaN,NaN
3,Region Syddanmark,2006,15-19 years,4.7,NaN,NaN,NaN
4,Region Midtjylland,2006,15-19 years,3.2,NaN,NaN,NaN
5,Region Nordjylland,2006,15-19 years,5.5,NaN,NaN,NaN
6,Province Byen København,2006,15-19 years,4.4,NaN,NaN,NaN
7,Province Københavns omegn,2006,15-19 years,4.1,NaN,NaN,NaN
8,Province Nordsjælland,2006,15-19 years,3.4,NaN,NaN,NaN
9,Province Bornholm,2006,15-19 years,2.4,NaN,NaN,NaN
